In [1]:
!pip install backtesting ta-lib seaborn matplotlib

In [1]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply, barssince
from backtesting.test import GOOG
import talib as ta
import seaborn as sns
import matplotlib.pyplot as plt

#print(GOOG)

/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-date

In [7]:
import mysql.connector
import pandas as pd
from mysql.connector import Error
conn = None
try:
    # Conéctate a la base de datos MySQL
    conn = mysql.connector.connect(
        host='localhost',
        database='backtesting',
        user='pruebas',
        password='contraseñaprueba'
    )

    if conn.is_connected():
        # Ejecuta la consulta para obtener los datos
        query = "SELECT * FROM btc_usdt"
        df_btc = pd.read_sql(query, conn)

        print("Datos obtenidos de la base de datos:")
        print(df_btc.head())

except Error as e:
    print(f"Error: {e}")
finally:
    if conn is not None and conn.is_connected():
        conn.close()
        print("Conexión a MySQL cerrada.")

#para cargar correctamente los datos a la estrategia las columnas deben estar con mayusculas
df_btc.columns = [col.capitalize() for col in df_btc.columns]
df_btc

Datos obtenidos de la base de datos:
            timestamp      open      high       low     close      volume
0 2024-04-25 06:00:00  64223.36  64391.99  64195.70  64288.00   363.60804
1 2024-04-25 07:00:00  64288.00  64379.85  63824.07  64017.99   951.17474
2 2024-04-25 08:00:00  64017.99  64176.86  63708.20  63948.78  1329.27338
3 2024-04-25 09:00:00  63948.78  64192.00  63809.13  63966.02   833.85634
4 2024-04-25 10:00:00  63966.03  63966.03  63337.25  63462.01  2418.87397
Conexión a MySQL cerrada.


/tmp/ipykernel_312378/798324656.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_btc = pd.read_sql(query, conn)


,Timestamp,Open,High,Low,Close,Volume
0,2024-04-25 06:00:00,64223.36,64391.99,64195.70,64288.00,363.60804
1,2024-04-25 07:00:00,64288.00,64379.85,63824.07,64017.99,951.17474
2,2024-04-25 08:00:00,64017.99,64176.86,63708.20,63948.78,1329.27338
3,2024-04-25 09:00:00,63948.78,64192.00,63809.13,63966.02,833.85634
4,2024-04-25 10:00:00,63966.03,63966.03,63337.25,63462.01,2418.87397
...,...,...,...,...,...,...
995,2024-06-05 17:00:00,71312.78,71704.00,71231.48,71628.98,1367.90189
996,2024-06-05 18:00:00,71628.98,71748.22,70895.15,71096.81,1778.53896
997,2024-06-05 19:00:00,71096.80,71398.73,70871.20,71279.95,937.98536
998,2024-06-05 20:00:00,71279.95,71357.68,71160.00,71196.02,699.46084


In [6]:
df_btc.columns = [col.capitalize() for col in df_btc.columns]
df_btc

,Timestamp,Open,High,Low,Close,Volume
0,2024-04-25 06:00:00,64223.36,64391.99,64195.70,64288.00,363.60804
1,2024-04-25 07:00:00,64288.00,64379.85,63824.07,64017.99,951.17474
2,2024-04-25 08:00:00,64017.99,64176.86,63708.20,63948.78,1329.27338
3,2024-04-25 09:00:00,63948.78,64192.00,63809.13,63966.02,833.85634
4,2024-04-25 10:00:00,63966.03,63966.03,63337.25,63462.01,2418.87397
...,...,...,...,...,...,...
995,2024-06-05 17:00:00,71312.78,71704.00,71231.48,71628.98,1367.90189
996,2024-06-05 18:00:00,71628.98,71748.22,70895.15,71096.81,1778.53896
997,2024-06-05 19:00:00,71096.80,71398.73,70871.20,71279.95,937.98536
998,2024-06-05 20:00:00,71279.95,71357.68,71160.00,71196.02,699.46084


In [5]:
def optim_func(series):

    if series["# Trades"]<10:
        return -1

    return series["Equity Final [$]"]/series["Exposure Time [%]"]

class RsiOscillator(Strategy):
    upper_bound = 70
    lower_bound = 30
    rsi_windows = 14

    def init(self):
        self.rsi = self.I(ta.RSI, self.data.Close, self.rsi_windows)

    def next(self):
        if crossover(self.rsi, self.upper_bound):
            self.position.close()
        elif crossover(self.lower_bound, self.rsi):
            self.buy()

bt = Backtest(df_btc, RsiOscillator, cash=10_000)

#stats = bt.run()
stats, heatmap = bt.optimize(
    upper_bound=range(50,85,1),
    lower_bound=range(10,45,1),
    rsi_windows=range(10,45,1),
    #maximize=optim_func,
    maximize='Sharpe Ratio',
    constraint= lambda param: param.upper_bound > param.lower_bound,
    return_heatmap=True
    #max_tries=100
)

#print(stats)

#hm = heatmap.groupby(["upper_bound","lower_bound"]).mean().unstack()
#sns.heatmap(hm)
#plt.show()
#plot_heatmaps(heatmap, agg="mean")

#print(hm)
bt.plot()

/tmp/ipykernel_212108/1474722984.py:22: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df_btc, RsiOscillator, cash=10_000)
/tmp/ipykernel_212108/1474722984.py:22: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df_btc, RsiOscillator, cash=10_000)
/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 42875 configurations.
  output = _optimize_grid()


GridPlot(id='p1283', ...)

# bt.run()
Start                     2004-08-19 00:00:00\
End                       2013-03-01 00:00:00\
Duration                   3116 days 00:00:00\
Exposure Time [%]                   29.329609\
Equity Final [$]                      15677.5\
Equity Peak [$]                       15835.7\
Return [%]                             56.775\
Buy & Hold Return [%]              703.458242\
Return (Ann.) [%]                    5.416737\
Volatility (Ann.) [%]               23.502544\
Sharpe Ratio                         0.230475\
Sortino Ratio                        0.386423\
Calmar Ratio                         0.110499\
Max. Drawdown [%]                  -49.020734\
Avg. Drawdown [%]                   -6.135224\
Max. Drawdown Duration     1603 days 00:00:00\
Avg. Drawdown Duration       94 days 00:00:00\
Trades                                   10\
Win Rate [%]                             80.0\
Best Trade [%]                      21.397983\
Worst Trade [%]                    -20.449092\
Avg. Trade [%]                       5.357299\
Max. Trade Duration         202 days 00:00:00\
Avg. Trade Duration          98 days 00:00:00\
Profit Factor                        3.862903\
Expectancy [%]                       5.995809\
SQN                                  1.227626\
_strategy                       RsiOscillator\
_equity_curve                          Equ...\
_trades                      Size  EntryBa...\
dtype: object

# bt.optimize(upper_bound = range(50,85,5),lower_bound = range(10,45,5),rsi_windows = range(10,30,2),maximize='Sharpe Ratio')
Start                     2004-08-19 00:00:00\
End                       2013-03-01 00:00:00\
Duration                   3116 days 00:00:00\
Exposure Time [%]                   62.662942\
Equity Final [$]                     45486.35\
Equity Peak [$]                      45573.15\
Return [%]                           354.8635\
Buy & Hold Return [%]              703.458242\
Return (Ann.) [%]                   19.448742\
Volatility (Ann.) [%]               32.629176\
Sharpe Ratio                         0.596054\
Sortino Ratio                        1.122836\
Calmar Ratio                          0.35036\
Max. Drawdown [%]                  -55.510789\
Avg. Drawdown [%]                   -4.037504\
Max. Drawdown Duration      548 days 00:00:00\
Avg. Drawdown Duration       34 days 00:00:00\
Trades                                   17\
Win Rate [%]                        76.470588\
Best Trade [%]                      41.689994\
Worst Trade [%]                    -27.637707\
Avg. Trade [%]                      11.346119\
Max. Trade Duration         240 days 00:00:00\
Avg. Trade Duration         119 days 00:00:00\
Profit Factor                        5.778866\
Expectancy [%]                      12.740143\
SQN                                  2.525661\
_strategy                 RsiOscillator(up...\
_equity_curve                           Eq...\
_trades                       Size  EntryB...\
dtype: object

In [6]:
def optim_func(series):

    if series["# Trades"]<10:
        return -1

    return series["Equity Final [$]"]/series["Exposure Time [%]"]

class RsiOscillator(Strategy):
    upper_bound = 70
    lower_bound = 30
    rsi_windows = 14

    def init(self):
        self.daily_rsi = self.I(ta.RSI, self.data.Close, self.rsi_windows)

        self.weekly_rsi = resample_apply(
            "W-FRI", ta.RSI, self.data.Close,self.rsi_windows
        )

    def next(self):
        if (crossover(self.daily_rsi, self.upper_bound) and self.weekly_rsi[-1]>self.upper_bound):
            self.position.close()
        elif (crossover(self.lower_bound, self.daily_rsi) and self.weekly_rsi[-1]<self.lower_bound):# and self.daily_rsi[-1]<self.weekly_rsi[-1]
            self.buy()

bt = Backtest(GOOG, RsiOscillator, cash=10_000)

stats = bt.run()
bt.plot()

/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Abriendo en una sesión existente del navegador


GridPlot(id='p2167', ...)

longs y shorts

In [7]:
def optim_func(series):

    if series["# Trades"]<10:
        return -1

    return series["Equity Final [$]"]/series["Exposure Time [%]"]

class RsiOscillator(Strategy):
    upper_bound = 75
    lower_bound = 35
    rsi_windows = 12

    def init(self):
        self.daily_rsi = self.I(ta.RSI, self.data.Close, self.rsi_windows)

        self.weekly_rsi = resample_apply(
            "W-FRI", ta.RSI, self.data.Close,self.rsi_windows
        )

    def next(self):
        if crossover(self.daily_rsi, self.upper_bound):
            if self.position.is_long:
                #print(self.position.size)
                #print(self.position.pl_pct)
                self.position.close()
                self.sell()

        elif crossover(self.lower_bound, self.daily_rsi):
            if self.position.is_short or not self.position:
                self.position.close()
                self.buy()

bt = Backtest(GOOG, RsiOscillator, cash=10_000)

stats = bt.run()
bt.plot()

/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
[212935:212935:0100/000000.048490:ERROR:zygote_linux.cc(675)] write: Broken pipe (32)


Abriendo en una sesión existente del navegador


GridPlot(id='p3422', ...)

# Gestion de riesgo

stop-loss y take-profit

In [8]:
def optim_func(series):

    if series["# Trades"]<10:
        return -1

    return series["Equity Final [$]"]/series["Exposure Time [%]"]

class RsiOscillator(Strategy):
    upper_bound = 75
    lower_bound = 35
    rsi_windows = 12

    def init(self):
        self.daily_rsi = self.I(ta.RSI, self.data.Close, self.rsi_windows)

        self.weekly_rsi = resample_apply(
            "W-FRI", ta.RSI, self.data.Close,self.rsi_windows
        )

    def next(self):

        price = self.data.Close[-1]

        if crossover(self.daily_rsi, self.upper_bound):
            self.position.close()

        elif crossover(self.lower_bound, self.daily_rsi):
            self.buy(tp=1.15*price,sl=0.95*price)

bt = Backtest(GOOG, RsiOscillator, cash=10_000)

stats = bt.run()
bt.plot()

/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
[213042:213042:0100/000000.333559:ERROR:zygote_linux.cc(675)] write: Broken pipe (32)


Abriendo en una sesión existente del navegador


GridPlot(id='p4677', ...)

tamaño de la orden

In [9]:
def optim_func(series):

    if series["# Trades"]<10:
        return -1

    return series["Equity Final [$]"]/series["Exposure Time [%]"]

class RsiOscillator(Strategy):
    upper_bound = 75
    lower_bound = 35
    rsi_windows = 12

    def init(self):
        self.daily_rsi = self.I(ta.RSI, self.data.Close, self.rsi_windows)

        self.weekly_rsi = resample_apply(
            "W-FRI", ta.RSI, self.data.Close,self.rsi_windows
        )

    def next(self):

        price = self.data.Close[-1]

        if crossover(self.daily_rsi, self.upper_bound):
            self.position.close()

        elif self.lower_bound > self.daily_rsi[-1]:
            #self.buy(size=1)
            self.buy(size=0.02)

bt = Backtest(GOOG, RsiOscillator, cash=10_000)

stats = bt.run()
bt.plot()

/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Abriendo en una sesión existente del navegador


GridPlot(id='p5932', ...)

funcion barssince()

In [10]:
def optim_func(series):

    if series["# Trades"]<10:
        return -1

    return series["Equity Final [$]"]/series["Exposure Time [%]"]

class RsiOscillator(Strategy):
    upper_bound = 70
    lower_bound = 30
    rsi_windows = 14

    def init(self):
        self.daily_rsi = self.I(ta.RSI, self.data.Close, self.rsi_windows)

    def next(self):

        price = self.data.Close[-1]

        if (self.daily_rsi[-1] > self.upper_bound and barssince(self.daily_rsi < self.upper_bound==3)):
            self.position.close()

        elif self.lower_bound > self.daily_rsi[-1]:
            #self.buy(size=1)
            self.buy(size=0.02)

bt = Backtest(GOOG, RsiOscillator, cash=10_000)

stats = bt.run()

print(stats['_trades'].to_string())
bt.plot()

Empty DataFrame
Columns: [Size, EntryBar, ExitBar, EntryPrice, ExitPrice, PnL, ReturnPct, EntryTime, ExitTime, Duration]
Index: []


/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/mjb/Escritorio/pruebaAPI-crp/env/lib/python3.8/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Abriendo en una sesión existente del navegador


GridPlot(id='p6860', ...)

Abriendo en una sesión existente del navegador


---
---

tutorial del sitio oficial de Backtesting

In [3]:
import pandas as pd
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting.test import SMA
from backtesting import Backtest

In [4]:
class SmaCross(SignalStrategy,
               TrailingStrategy):
    n1 = 10
    n2 = 25
    
    def init(self):
        # In init() and in next() it is important to call the
        # super method to properly initialize the parent classes
        super().init()
        
        # Precompute the two moving averages
        sma1 = self.I(SMA, self.data.Close, self.n1)
        sma2 = self.I(SMA, self.data.Close, self.n2)
        
        # Where sma1 crosses sma2 upwards. Diff gives us [-1,0, *1*]
        signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
        signal = signal.replace(-1, 0)  # Upwards/long only
        
        # Use 95% of available liquidity (at the time) on each order.
        # (Leaving a value of 1. would instead buy a single share.)
        entry_size = signal * .95
                
        # Set order entry sizes using the method provided by 
        # `SignalStrategy`. See the docs.
        self.set_signal(entry_size=entry_size)
        
        # Set trailing stop-loss to 2x ATR using
        # the method provided by `TrailingStrategy`
        self.set_trailing_sl(2)

In [8]:
bt = Backtest(df_btc, SmaCross, commission=.002)

bt.run()
bt.plot()

/tmp/ipykernel_312378/753972421.py:1: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df_btc, SmaCross, commission=.002)
/tmp/ipykernel_312378/753972421.py:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df_btc, SmaCross, commission=.002)


GridPlot(id='p1308', ...)

Abriendo en una sesión existente del navegador
